In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers datasets torch trl peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
)
from trl import SFTTrainer
import torch
from peft import LoraConfig, get_peft_model

dataset = load_dataset("csv", data_files="/content/drive/MyDrive/R1-1.5B/Microbiology.csv")

def format_dataset(examples):
    texts = []
    for instruction, response in zip(examples["Instruction"], examples["Output"]):
        formatted_text = f"### Instruction:\n{instruction}\n\n### Response:\n{response}"
        texts.append(formatted_text)
    return {"text": texts}

dataset = dataset.map(format_dataset, batched=True, remove_columns=["Instruction", "Output"])

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
# model = model.to('cuda')

## Configure LoRA
peft_config = LoraConfig(
    r=16,                   # Rank of the low-rank matrices
    lora_alpha=32,          # Scaling factor
    lora_dropout=0.1,       # Dropout for LoRA layers
    bias="none",            # No bias for LoRA
    task_type="CAUSAL_LM",  # Task type
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]  # Target modules for LoRA
)
model = get_peft_model(model, peft_config)

## Define training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/R1-1.5B/results",  # Directory to save results
    num_train_epochs=120,           # Number of training epochs
    per_device_train_batch_size=2,  # Batch size per device
    per_device_eval_batch_size=2,   # Evaluation batch size
    gradient_accumulation_steps=2,  # Gradient accumulation steps
    gradient_checkpointing=False,   # Disable gradient checkpointing for debugging
    optim="adamw_torch",            # Optimizer
    learning_rate=1e-4,             # Learning rate
    warmup_ratio=0.1,               # Warmup ratio
    fp16=True,                      # Use mixed precision (FP16)
    logging_steps=200,              # Log every 10 steps
    save_strategy="steps",          # Save model at specific steps
    save_steps=200,                 # Save every 200 steps
    save_total_limit=2,             # Save steps
    evaluation_strategy="steps",    # Evaluate at specific steps
    eval_steps=200,                 # Evaluate every 200 steps
    eval_accumulation_steps=1,      # Accumulate evaluation steps
    load_best_model_at_end=True,    # Load the best model at the end
    metric_for_best_model="eval_loss",  # Metric for best model
    greater_is_better=False,        # Lower eval_loss is better
    remove_unused_columns=True,     # Remove unused columns
    report_to="none",               # Disable external logging
)

## Split Train and Test
split_dataset = dataset["train"].train_test_split(test_size=150, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

## Initialize the trainer
trainer = SFTTrainer(
    model = model,
    args = training_args,
    # train_dataset=dataset["train"],
    # eval_dataset=dataset["train"].select(range(120)),  # Small evaluation set
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    tokenizer = tokenizer,
)

## Train the model
# trainer.train()
checkpoint = '/content/drive/MyDrive/R1-1.5B/results/checkpoint-27700'
trainer.train(resume_from_checkpoint=checkpoint)

## Save the model
trained_model = "/content/drive/MyDrive/R1-1.5B/fine-tuned-deepseek-r1-1.5b"
model.save_pretrained(trained_model)
tokenizer.save_pretrained(trained_model)
# # trainer.save_model(trained_model)
# trainer.model.save_pretrained(trained_model)
# trainer.tokenizer.save_pretrained(trained_model)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-15-590a351adf3a>:70: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:3441: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pyt

Step,Training Loss,Validation Loss
27800,0.034700,0.981139
28000,0.033000,0.974706
28200,0.035200,0.974130
28400,0.034400,0.974533
28600,0.032400,0.974873
28800,0.039400,0.975019
29000,0.035000,0.981379


### Inference

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_path = "/content/drive/MyDrive/R1-1.5B/fine-tuned-deepseek-r1-1.5b"
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    # device_map="auto"
)
model = model.to('cuda')
tokenizer = AutoTokenizer.from_pretrained(model_path)

def generate_text(prompt, max_new_tokens=512):
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.5,
            top_k=50,
            top_p=0.9,
            use_cache=True
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)

prompt = "ဆေးရုံတွင်း ကူးစက်ရောဂါ များကြောင့် စီးပွားရေးအပေါ် သက်ရောက်မှုများကို ဖော်ပြပါ"
output = generate_text(prompt)
print(output)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


ဆေးရုံတွင်း ကူးစက်ရောဂါ များကြောင့် စီးပွားရေးအပေါ် သက်ရောက်မှုများကို ဖော်ပြပါသည် ။

### စားသုံးခြင်း
- ကြွက်သားများသို့ မှိုချိန်ကြွက်ကို စွမ်းအင်အများစု ရှိသည် ။
- ကြွက်သားများသို့ ရောက်ရှိခြင်း မှတစ်ဆင့် ကူးစက်နိုင်သည် ။
- ကြွက်သားများသို့ ပျက်စီးမှု မျိုးစိတ်များ ရှိသည် ။
- ကြွက်သားများသို့ ရောက်ရှိခြင်း မှတစ်ဆင့် ကူးစက်နိုင်သည် ။
- ကြွက်သားများသို့ ရောက်ရှိခြင်း မှတစ်ဆင့် ကူးစက်နိုင်သည် ။
- ကြွက်သားများသို့


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "/content/drive/MyDrive/R1-1.5B/fine-tuned-deepseek-r1-1.5b"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16)
model = model.to('cuda')

instruction = "Selective agar ဆိုတာ ဘာလဲ ရှင်းပြပါ"
input_text = f"### Instruction:\n{instruction}\n\n### Response:"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=512,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        do_sample = True,
        use_cache=True,
        num_return_sequences=1
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
response = response.split("### Response:")[-1].strip()
print(response)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Selective agar သည် မှတ်တိမီerobita ဘက်တီးရီးယား အမျိုးအစား ၏ တုံ့ရှိ ပုံသဏ္ဍာန် ကူးစက်ပျံ့နှံ့သော အစားအစာ များကို မျှဝေသုံးစွဲခြင်း ဖြစ်သည် ။ ၎င်းတွင် Gram-positive ဘက်တိမီerobita ဘက်တိမီerobita အတွက် မွေးမြူသည် ။ Gram-positive ဘက်တိမီerobita အသုံးပြုရေးလျှင် selective agar များ ဖြစ်ပေါ်စေပြီး Gram-negative ဘက်တိမီerobita အသုံးပြုရေး ထုတ်လွှက်သည် ။ အဖျားဆုံး တွင် ပြင်းထန်သော ဘက်တိမီerobita များကို တွေ�
